## 1. Import necessary module

In [1]:
from scipy.optimize import milp, LinearConstraint
from qiskit_optimization import QuadraticProgram
from qiskit import QiskitError
import numpy as np
from qiskit_optimization.algorithms import GurobiOptimizer


TypeError: solve() missing 2 required positional arguments: 'self' and 'problem'

In [2]:
%load_ext autoreload
%autoreload 2

## 2. Create a demo problem

### Example 1


$${\rm max} \ x_1+x_2+2x_3 \\

-x_1+x_2+x_3 \leq 1 \\ 3x_1+2x_2 - x_3 \leq 12 \\ x_1+x_2 +x_3 \leq 10 \\ 2x_1+3x_2 + 3x_3 \geq 12 \\ 

x_1, x_2 \in \mathcal{Z}
$$

Let's solve it with the milp solver!

In [14]:
## constant value in objective

c = -np.array([1, 2, 2])  
A = np.array([[-1, 1, 1], [3, 2, -1], [1, 1, 1], [2, 3, 3]])
b_u = np.array([1, 12, 10, np.inf])
b_l = np.array([-np.inf, -np.inf, -np.inf, 12])

constraints = LinearConstraint(A, b_l, b_u)
integrality = np.array([1, 1, 0])
#integrality = np.zeros(3)
res = milp(c=c, constraints=constraints, integrality=integrality)

print(res)

            fun: -15.0
        message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
 mip_dual_bound: -15.0
        mip_gap: 0.0
 mip_node_count: 1
         status: 0
        success: True
              x: array([5., 0., 5.])


We see the MILP solver tells us the object function is maximized when $x_1 = 5, x_2 = 0, x_3 = 5$.

As the next step, let's create a corresponding `QuadraticProgram` class in `qiskit_optimization`.

In [17]:
## Consider the possibility of using sparse matrix(to_dict)
## 1e20=inf
qp = QuadraticProgram('example-1')
qp.integer_var(name='x1')
qp.integer_var(name='x2')
qp.continuous_var(name='x3')
#qp.binary_var_list(2000)
qp.maximize(linear={'x1': 1, 'x2': 1, 'x3': 2})
#qp.minimize(linear={'x2': 1})
qp.linear_constraint({'x1': -1, 'x2': 1, 'x3': 1},'<=',1)
qp.linear_constraint({'x1': 1, 'x2': 1, 'x3': 1},'<=',10)
qp.linear_constraint({'x1': 3, 'x2': 2, 'x3': -1},'<=',12)
qp.linear_constraint({'x1': 2, 'x2': 3, 'x3': 3},'>=',12)

print(qp.prettyprint())

Problem name: example-1

Maximize
  x1 + x2 + 2*x3

Subject to
  Linear constraints (4)
    -x1 + x2 + x3 <= 1  'c0'
    x1 + x2 + x3 <= 10  'c1'
    3*x1 + 2*x2 - x3 <= 12  'c2'
    2*x1 + 3*x2 + 3*x3 >= 12  'c3'

  Integer variables (2)
    0 <= x1
    0 <= x2

  Continuous variables (1)
    0 <= x3



We solve the `qp(QuadraticProgram)` with the Gurobi solver.

In [18]:
gurobi_result = GurobiOptimizer().solve(qp)
print(gurobi_result.prettyprint())

objective function value: 15.0
variable values: x1=5.0, x2=-0.0, x3=5.0
status: SUCCESS


## 3. Build a Converter

The converter should be able to extract necessary information from the `QuadraticProblem` class and input it to the milp solver.

In [34]:
## First, Check if qp is linear
## Check objective is linear
from types import NoneType


if qp.objective.quadratic.to_dict()!={}:
    raise QiskitError('Obejective function is not linear!')

## Check constrains are linear
if qp.quadratic_constraints:
    raise QiskitError('Constraints are not linear!')

## Sense: 1 for minimization and -1 for maximization 
sense = qp.objective.sense.value

## cost function for milp solver
c_qp = qp.objective.linear.to_array() * sense  

## constraints for milp solver
for i, constraint in enumerate(qp.linear_constraints):

    constraint_array = constraint.linear.to_array()
    constraint_sense = constraint.sense.value # 0 for leq, 1 for geq
    constraint_value = constraint.rhs
    if i==0:
        A_qp = constraint_array
        bl_qp = np.array([constraint_value if constraint_sense==1 else -np.inf])
        bu_qp = np.array([constraint_value if constraint_sense==0 else np.inf])
    else:
        A_qp = np.vstack((A_qp,constraint_array))  ## Not efficient. Use list directly.
        bl_qp = np.append(bl_qp, [constraint_value if constraint_sense==1 else -np.inf])
        bu_qp = np.append(bu_qp, [constraint_value if constraint_sense==0 else np.inf])

constraints = LinearConstraint(A_qp, bl_qp, bu_qp)

## integrity for milp solver
## Use Enum and VarType to compare
integrality = np.array([ 1 if variable.vartype.value==2 else 0 for variable in qp.variables]) ## check on other vartype!

raw_res = milp(c=c_qp, constraints=constraints, integrality=integrality)

print(raw_res)



            fun: -15.0
        message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
 mip_dual_bound: -15.0
        mip_gap: 0.0
 mip_node_count: 1
         status: 0
        success: True
              x: array([5., 0., 5.])


In [ ]:
## Sparse converter based on to_dict

In [37]:
qp.linear_constraints.sense

AttributeError: 'QuadraticProgram' object has no attribute 'constraints'

In [29]:
A = qp.get_linear_constraint(0).linear.to_array()
B = qp.get_linear_constraint(1).linear.to_array()
A = np.vstack((A, B))
A

array([[-1.,  1.,  1.],
       [ 3.,  2., -1.]])

In [62]:
qp.get_linear_constraint(0).sense

<ConstraintSense.LE: 0>

In [26]:
integrality = np.array([ 1 if variable.vartype.value==2 else 0 for variable in qp.variables])